In [25]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

In [26]:
df_combined = pd.read_csv('df_combined.csv')

In [27]:
df_combined

,date,price_bitcoin,price_gold,price_sp500,change_bitcoin,change_gold,vix,interest_rate,cpi,inflation
0,2015-01-02,315,1186,2058,0.10,0.18,17.79,0.01,234.82,0.76
1,2015-01-05,275,1204,2021,3.82,1.50,19.92,0.00,234.84,0.76
2,2015-01-06,282,1219,2003,2.70,1.28,21.12,0.02,234.84,0.76
3,2015-01-07,291,1211,2026,3.21,-0.71,19.31,0.02,234.84,0.76
4,2015-01-08,283,1208,2062,-2.97,-0.18,17.01,0.02,234.85,0.76
...,...,...,...,...,...,...,...,...,...,...
2288,2024-02-05,42697,2043,4943,0.27,-0.53,13.67,5.23,307.54,3.35
2289,2024-02-06,43088,2051,4954,0.91,0.42,13.06,5.22,307.56,3.35
2290,2024-02-07,44340,2052,4995,2.91,0.01,12.83,5.22,307.58,3.35
2291,2024-02-08,45293,2048,4998,2.15,-0.19,12.79,5.22,307.60,3.35


Primero, leemos los datos del archivo CSV y nos aseguramos de insertar las fechas únicas en la tabla dates.

In [ ]:
# Conexión a la base de datos
bd = "financial_analysis"
password = "123456"  # Tu contraseña
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Leer el archivo CSV
df_combined = pd.read_csv('df_combined.csv')

# Extraer las fechas únicas del CSV
df_dates = pd.DataFrame(df_combined['date'].unique(), columns=['date'])

# Insertar las fechas en la tabla 'dates'
df_dates.to_sql('dates', con=engine, if_exists='append', index=False)

Después de insertar las fechas, hacemos un merge del archivo CSV con la tabla dates para asignar el id_date correcto en cada fila del CSV.

In [ ]:
# Cargar las fechas desde la tabla 'dates' para obtener el id_date
dates_from_db = pd.read_sql('SELECT * FROM dates', con=engine)

# Asegurarse de que las fechas estén en el mismo formato
df_combined['date'] = pd.to_datetime(df_combined['date']).dt.date
dates_from_db['date'] = pd.to_datetime(dates_from_db['date']).dt.date

# Hacer el merge entre el CSV y la tabla 'dates' usando la columna 'date'
df_merged = pd.merge(df_combined, dates_from_db, how='left', on='date')

# Eliminar la columna 'id_date_x' y renombrar 'id_date_y' a 'id_date'
df_merged.drop(columns=['id_date_x'], inplace=True)
df_merged.rename(columns={'id_date_y': 'id_date'}, inplace=True)

Una vez hecho el merge y corregido el id_date, dividimos los datos entre los activos y los factores económicos y los insertamos en las tablas correspondientes.

In [ ]:
# Dividir el DataFrame en las partes necesarias con el id_date ya corregido
assets_df = df_merged[['id_date', 'price_bitcoin', 'price_gold', 'price_sp500', 'change_bitcoin', 'change_gold']]
economic_factors_df = df_merged[['id_date', 'vix', 'interest_rate', 'cpi', 'inflation']]

# Insertar los datos en las tablas correspondientes
assets_df.to_sql('assets', con=engine, if_exists='append', index=False)
economic_factors_df.to_sql('economic_factors', con=engine, if_exists='append', index=False)